In [1]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception, inception_v3

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

Using TensorFlow backend.


In [2]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

emsemble a few augmentation training data

In [3]:
base_model_x = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
model_x = add_preprocess(base_model_x, xception.preprocess_input)

In [4]:
base_model_i = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')
model_i = add_preprocess(base_model_i, inception_v3.preprocess_input)

In [5]:
batches = image.ImageDataGenerator().flow_from_directory(data_dir+'/train', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_val = image.ImageDataGenerator().flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_test = image.ImageDataGenerator().flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)
nb_batches_test = math.ceil(batches_test.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [6]:
# bf_val_x = model_x.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_x', bf_val_x)
bf_val_x = np.load(data_dir+'/results/bf_val_x.npy')

# bf_x_test = model_x.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_x_test', bf_x_test)
bf_x_test = np.load(data_dir+'/results/bf_x_test.npy')

In [18]:
# bf_val_i = model_i.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_i', bf_val_i)
bf_val_i = np.load(data_dir+'/results/bf_val_i.npy')

# bf_i_test = model_i.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_i_test', bf_i_test)
bf_i_test = np.load(data_dir+'/results/bf_i_test.npy')

In [8]:
gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               shear_range=0.15, 
                               zoom_range=0.1, 
                               channel_shift_range=10., 
                               horizontal_flip=True)

In [45]:
preds = []
nb_runs = 20
for i in range(nb_runs):
    print("i:", i)
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    y = to_categorical(batches.classes)
    bf_x = model_x.predict_generator(batches, steps=nb_batches, verbose=1)
    
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    bf_i = model_i.predict_generator(batches, steps=nb_batches, verbose=1)
    
    lm = Sequential([Dense(120, activation='softmax', input_shape=(2048+2048,))])
    lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    lm.fit(np.hstack([bf_x, bf_i]), y, epochs=15, batch_size=nb_batches, 
           validation_data=(np.hstack([bf_val_x, bf_val_i]), y_val))
    
    pred = lm.predict(np.hstack([bf_x_test, bf_i_test]), batch_size=batch_size, verbose=1)
    preds.append(pred)

i: 0
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 125s   
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 112s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 2.1057 - acc: 0.6459 - val_loss: 0.7992 - val_acc: 0.8805
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 0.5950 - acc: 0.8927 - val_loss: 0.4584 - val_acc: 0.8935
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.3846 - acc: 0.9087 - val_loss: 0.3579 - val_acc: 0.9020
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.2913 - acc: 0.9237 - val_loss: 0.3167 - val_acc: 0.9090
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.2301 - acc: 0.9385 - val_loss: 0.3010 - val_acc: 0.9085
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.1921 - acc: 0.9479 - val_loss: 0.2873 - val_acc: 0.9120
Epoch 7

8222/8222 [==============================] - 0s - loss: 0.1635 - acc: 0.9554 - val_loss: 0.2872 - val_acc: 0.9075
Epoch 8/15
8222/8222 [==============================] - 0s - loss: 0.1399 - acc: 0.9646 - val_loss: 0.2847 - val_acc: 0.9100
Epoch 9/15
8222/8222 [==============================] - 0s - loss: 0.1202 - acc: 0.9696 - val_loss: 0.2867 - val_acc: 0.9040
Epoch 10/15
8222/8222 [==============================] - 0s - loss: 0.1028 - acc: 0.9746 - val_loss: 0.2757 - val_acc: 0.9130
Epoch 11/15
8222/8222 [==============================] - 0s - loss: 0.0906 - acc: 0.9796 - val_loss: 0.2708 - val_acc: 0.9155
Epoch 12/15
8222/8222 [==============================] - 0s - loss: 0.0779 - acc: 0.9836 - val_loss: 0.2815 - val_acc: 0.9075
Epoch 13/15
8222/8222 [==============================] - 0s - loss: 0.0694 - acc: 0.9848 - val_loss: 0.2823 - val_acc: 0.9085
Epoch 14/15
8222/8222 [==============================] - 0s - loss: 0.0600 - acc: 0.9882 - val_loss: 0.2836 - val_acc: 0.9075
Epoch 

 9472/10357 [==========================>...] - ETA: 0si: 15
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 125s   
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 113s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 2.1067 - acc: 0.6474 - val_loss: 0.8138 - val_acc: 0.8670
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 0.5919 - acc: 0.8918 - val_loss: 0.4592 - val_acc: 0.8945
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.3728 - acc: 0.9123 - val_loss: 0.3586 - val_acc: 0.9000
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.2844 - acc: 0.9291 - val_loss: 0.3248 - val_acc: 0.9035
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.2282 - acc: 0.9414 - val_loss: 0.2999 - val_acc: 0.9040
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.1888 - a

In [46]:
pred_ensemble = np.stack(preds).mean(axis=0)

### predict

In [47]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [48]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred_ensemble]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [49]:
description = 'xception_inception_ensemble_%d_data_aug' % nb_runs
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [50]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [51]:
!kg submit $submission_file_name -m $description

0.26652
